<a href="https://colab.research.google.com/github/lindapu-1/30DaysReadmission/blob/main/LSTM%EF%BC%8CGNN%EF%BC%8CMetric_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import zipfile
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
zip_data_path = "/gdrive/MyDrive/all_data_3612.zip"

!mkdir all_data_3612
extract_path = '/content/all_data_3612'

with zipfile.ZipFile(zip_data_path, 'r') as zip_ref:
   zip_ref.extractall(extract_path)

Mounted at /gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/all_data_3612/ehr_preprocessed_seq_by_day_cat_embedding.pkl', 'rb') as f:
  data = pickle.load(f)

data.keys()

dict_keys(['feat_dict', 'feature_cols', 'cat_idxs', 'cat_dims', 'demo_cols', 'icd_cols', 'lab_cols', 'med_cols'])

In [ ]:
data['feat_dict']['12703486_29751691']

array([[42,  1,  4, ...,  0,  0,  0],
       [42,  1,  4, ...,  0,  0,  0],
       [42,  1,  4, ...,  0,  0,  0],
       ...,
       [42,  1,  4, ...,  0,  0,  0],
       [42,  1,  4, ...,  0,  0,  0],
       [42,  1,  4, ...,  0,  0,  0]])

In [ ]:
data['feat_dict']['14329220_29149180']

array([[72,  1,  6, ...,  0,  0,  1],
       [72,  1,  6, ...,  0,  0,  1],
       [72,  1,  6, ...,  0,  0,  1],
       ...,
       [72,  1,  6, ...,  0,  0,  1],
       [72,  1,  6, ...,  0,  0,  1],
       [72,  1,  6, ...,  0,  0,  1]])

In [ ]:
import pandas as pd

df_train = pd.read_csv("/content/all_data_3612/train.csv")
df_valid = pd.read_csv("/content/all_data_3612/valid.csv")
df_test = pd.read_csv("/content/all_data_3612/test.csv")

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, dataframe, feat_dict):
        self.df = dataframe.drop_duplicates(subset=['id'], keep='first')
        self.feat_dict = feat_dict
        self.mean, self.std = self.compute_stats()

    def compute_stats(self):
        # Combine only the selected features into a large array
        selected_features = torch.cat([torch.tensor(self.feat_dict[id][:, :], dtype=torch.float32) for id in self.df['id']])
        # Calculate mean and standard deviation for these features
        mean = selected_features.mean(dim=0)
        std = selected_features.std(dim=0)
        return mean, std

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Select only the relevant features
        features = torch.tensor(self.feat_dict[row['id']][:, ], dtype=torch.float32)
        # Check if the column exists, otherwise assign -1
        label = row['readmitted_within_30days'] if 'readmitted_within_30days' in self.df.columns else -1
        # Apply normalization to the selected features
        features = (features - self.mean) / (self.std + 1e-6)  # To avoid division by zero
        return features, torch.tensor(label, dtype=torch.long)

# Example usage
train_dataset = CustomDataset(df_train, data['feat_dict'])
valid_dataset = CustomDataset(df_valid, data['feat_dict'])
test_dataset = CustomDataset(df_test, data['feat_dict'])

In [ ]:
train_dataset[0][0].shape

torch.Size([13, 171])

In [ ]:
import torch

# 假设 train_dataset 和 valid_dataset 是您的PyTorch Dataset对象

def extract_features_and_labels(dataset):
    features_list = []
    labels_list = []

    for features, label in dataset:
        # 假设 features 是一个 (time_steps, num_features) 形状的张量
        # 对时间维度进行平均
        # features_mean = features.mean(dim=0)  # 对时间维度取平均值
        features_mean = features.mean(dim=0)  # 对时间维度取平均值
        features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储
        labels_list.append(label.item())  # 存储标签

    # 将列表转换为NumPy数组
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    return features_array, labels_array

# 提取训练和测试特征
train_features, train_labels = extract_features_and_labels(train_dataset)
valid_features, valid_labels = extract_features_and_labels(valid_dataset)
test_features, test_labels = extract_features_and_labels(test_dataset)

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

# 假设 train_features, test_features 是训练和测试特征数据
# train_labels 是训练数据的标签

# 用于存储所有测试点的预测概率
predicted_probabilities_list = []

# 对于测试数据中的每个点
from tqdm import tqdm
for test_point in tqdm(valid_features):
    # 计算与训练集中每个点的距离
    distances = np.linalg.norm(train_features - test_point, axis=1)
    nearest_indices = np.argsort(distances)[:20]
    nearest_features = train_features[nearest_indices]
    nearest_labels = train_labels[nearest_indices]

    # 检查是否只有一个类别
    unique_labels = np.unique(nearest_labels)
    if len(unique_labels) == 1:
        # 如果只有一个类别，预测概率为1.0或0.0，取决于该类别
        predicted_probability = 1.0 if unique_labels[0] == 1 else 0.0
    else:
        # 如果有多个类别，正常训练和预测
        svm_model = SVC(kernel='rbf', probability=True)
        svm_model.fit(nearest_features, nearest_labels)
        predicted_probability = svm_model.predict_proba(test_point.reshape(1, -1))[0, 1]

    predicted_probabilities_list.append(predicted_probability)

# 计算整个测试集的ROC-AUC
roc_auc = roc_auc_score(valid_labels, predicted_probabilities_list)
print("ROC-AUC for the entire test set:", roc_auc)


100%|██████████| 2325/2325 [00:12<00:00, 189.00it/s]

ROC-AUC for the entire test set: 0.6504714802653283


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# 初始化KNN模型
knn = KNeighborsClassifier(n_neighbors=300, weights = "distance")

# 训练模型
knn.fit(train_features, train_labels)

# 预测测试集的概率
predicted_probabilities = knn.predict_proba(valid_features)[:, 1]  # 获取正类的概率

# 计算整个测试集的ROC-AUC
roc_auc = roc_auc_score(valid_labels, predicted_probabilities)
print("ROC-AUC for the entire test set:", roc_auc)



ROC-AUC for the entire test set: 0.7494322190023295


In [ ]:
def compute_KNN(n = 1000):
    # 初始化KNN模型
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import roc_auc_score

    knn = KNeighborsClassifier(n_neighbors=n, weights = "distance")

    # 训练模型
    knn.fit(train_features, train_labels)

    # 预测测试集的概率
    predicted_probabilities = knn.predict_proba(valid_features)[:, 1]  # 获取正类的概率
    # 计算整个测试集的ROC-AUC
    roc_auc = roc_auc_score(valid_labels, predicted_probabilities)
    print("ROC-AUC for the entire test set:", roc_auc)

    predicted_probabilities_test = knn.predict_proba(test_features)[:, 1]  # 获取正类的概率

    return predicted_probabilities.tolist() + predicted_probabilities_test.tolist()


In [ ]:
valid_ids = df_valid['id'].tolist()
test_ids = df_test['id'].tolist()
combined_ids = valid_ids + test_ids
# Removing adjacent duplicates
unique_ids = [combined_ids[i] for i in range(len(combined_ids)) if i == 0 or combined_ids[i] != combined_ids[i-1]]
dict_outputDF = {
    "id": unique_ids,
    "subset": ["valid"] * len(valid_labels)+["test"]*len(test_labels),
    "label": valid_labels.tolist()+test_labels.tolist(),
    "knn_n=100": compute_KNN(n=100),
    "knn_n=500": compute_KNN(n=500),
    "knn_n=1000": compute_KNN(n=1000),
}

ROC-AUC for the entire test set: 0.7308248620831408
ROC-AUC for the entire test set: 0.742557310989195
ROC-AUC for the entire test set: 0.7464486725978106


In [ ]:
x = compute_KNN(n=50)

ROC-AUC for the entire test set: 0.7096317226538909


In [ ]:
import pandas as pd
df = pd.DataFrame(dict_outputDF)
df.to_csv('/content/gdrive/MyDrive/KNN.csv', index=False)

OSError: ignored

# Metric Learning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualEmbeddingNet(nn.Module):
    def __init__(self, num_features, embedding_dim=64):
        super(ResidualEmbeddingNet, self).__init__()
        self.fc1 = nn.Linear(num_features, 32)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(32, embedding_dim)
        # 如果输入特征维度与输出嵌入维度不同，需要一个额外的转换层
        if num_features != embedding_dim:
            self.shortcut = nn.Linear(num_features, embedding_dim)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        out += identity  # 添加残差连接
        return out

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positives=None, negatives=None):
        # 初始化损失为0
        loss = torch.tensor(0.0, device=anchor.device)

        # 计算正样本的平均距离
        if positives is not None and len(positives) > 0:
            distance_positives = [(anchor - p).pow(2).sum(1) for p in positives]
            average_distance_positive = torch.stack(distance_positives).mean(0)

        # 计算负样本的平均距离
        if negatives is not None and len(negatives) > 0:
            distance_negatives = [(anchor - n).pow(2).sum(1) for n in negatives]
            average_distance_negative = torch.stack(distance_negatives).mean(0)

        # 根据正负样本的存在情况计算损失
        if positives is not None and negatives is not None and len(positives) > 0 and len(negatives) > 0:
            loss = F.relu(average_distance_positive + self.margin)
        elif positives is not None and len(positives) > 0:
            loss = average_distance_positive
        elif negatives is not None and len(negatives) > 0:
            # loss = -average_distance_negative + self.margin
            loss = 0

        return loss.mean()


In [ ]:
model = ResidualEmbeddingNet(num_features=171)
loss_fn = TripletLoss(margin=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Including L1 penalty
l1_lambda = 0.001  # Adjust this value for the strength of the L1 penalty


# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare data for Nearest Neighbors and move to device
train_features = torch.tensor(train_features).to(device)
train_labels = torch.tensor(train_labels).to(device)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    # Compute nearest neighbors
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=200, algorithm='auto').fit(train_features.cpu().numpy())
    distances, indices = nbrs.kneighbors(train_features.cpu().numpy())
    from tqdm import trange
    for i in trange(train_features.size(0)):
        anchor = train_features[i].unsqueeze(0)
        neighbors = train_features[indices[i]]

        # Create positive and negative masks based on labels
        positive_mask = train_labels[indices[i]] == train_labels[i]
        negative_mask = ~positive_mask

        # Extract positives and negatives
        positives = neighbors[positive_mask]
        negatives = neighbors[negative_mask]

        # Get embeddings from the model
        anchor_embedding = model(anchor)
        positive_embeddings = model(positives) if positives.size(0) > 0 else None
        negative_embeddings = model(negatives) if negatives.size(0) > 0 else None

        # Compute triplet loss
        triplet_loss = loss_fn(anchor_embedding, positive_embeddings, negative_embeddings)

        # Calculate L1 penalty (sum of absolute values of all parameters)
        l1_penalty = sum(p.abs().sum() for p in model.parameters())

        # Total loss = Triplet loss + L1 penalty
        total_loss = triplet_loss + l1_lambda * l1_penalty

        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss / train_features.size(0)}")

100%|██████████| 9271/9271 [04:42<00:00, 32.83it/s]

Epoch 1, Loss: 1.8745197181163669


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

def evaluate_with_knn(model, n_neighbors=100):
    model.eval()  # Set the model to evaluation mode

    # Move model to the same device as the data

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Extract embeddings from the model
    with torch.no_grad():
        train_embeddings = model(torch.tensor(train_features).to(device)).cpu().numpy()
        valid_embeddings = model(torch.tensor(valid_features).to(device)).cpu().numpy()

    # Initialize and train KNN classifier
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights="distance")
    knn.fit(train_embeddings, train_labels.cpu().numpy())

    # Predict probabilities on the validation set
    predicted_probabilities = knn.predict_proba(valid_embeddings)[:, 1]

    # Compute ROC-AUC
    roc_auc = roc_auc_score(valid_labels, predicted_probabilities)
    print("ROC-AUC for the validation set:", roc_auc)

# Example of using the function
evaluate_with_knn(model, n_neighbors=50)


<ipython-input-151-948e6d6e1756>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_embeddings = model(torch.tensor(train_features).to(device)).cpu().numpy()


ROC-AUC for the validation set: 0.6554260831612017


# GNN

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00


In [ ]:
# Rest of your code with some modifications
import torch
from torch_geometric.data import Data
from sklearn.neighbors import kneighbors_graph
import numpy as np

# Combine features and labels from all sets
all_features = torch.cat((torch.tensor(train_features), torch.tensor(valid_features), torch.tensor(test_features)), 0)
all_labels = torch.cat((torch.tensor(train_labels), torch.tensor(valid_labels), torch.tensor(test_labels)), 0)

# Create a graph
A = kneighbors_graph(all_features.numpy(), n_neighbors=20, mode='distance', include_self=True)
edges = torch.tensor(np.array(A.nonzero()), dtype=torch.long)
data = Data(x=all_features, edge_index=edges, y=all_labels)
num_train = len(train_labels)
num_valid = len(valid_labels)
num_test = len(test_labels)
num_nodes = all_features.shape[0]

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
valid_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[:num_train] = True
valid_mask[num_train:num_train + num_valid] = True
test_mask[num_train + num_valid:num_train + num_valid + num_test] = True

data.train_mask = train_mask
data.valid_mask = valid_mask
data.test_mask = test_mask

In [ ]:
data

Data(x=[11596, 171], edge_index=[2, 1159600], y=[11596], train_mask=[11596], test_mask=[11596])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class DeepGCN(torch.nn.Module):
    def __init__(self, num_features, num_classes, dropout_rate=0.5):
        super(DeepGCN, self).__init__()
        self.conv1 = GCNConv(num_features, 32)
        self.conv2 = GCNConv(32, 32)
        self.conv3 = GCNConv(32, 32)  # Added missing conv3 layer
        self.conv4 = GCNConv(32, 32)
        self.conv5 = GCNConv(32, num_classes)
        self.dropout_rate = dropout_rate

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x1 = F.relu(self.conv1(x, edge_index))  # First layer
        x = F.dropout(x1, p=self.dropout_rate, training=self.training)

        x2 = F.relu(self.conv2(x, edge_index)) + x  # Second layer
        x = F.dropout(x2, p=self.dropout_rate, training=self.training)

        x3 = F.relu(self.conv3(x, edge_index)) + x  # Third layer with residual connection from first layer
        x = F.dropout(x3, p=self.dropout_rate, training=self.training)

        x4 = F.relu(self.conv4(x, edge_index)) + x  # Fourth layer with residual connection from second layer
        x = F.dropout(x4, p=self.dropout_rate, training=self.training)

        x = self.conv5(x, edge_index)  # Output layer
        return F.log_softmax(x, dim=1)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
NUM_FEATURES = 171
NUM_CLASSES = 2
NUM_EPOCHS = 500

# 将模型移动到指定的设备（GPU或CPU）
model = DeepGCN(num_features=NUM_FEATURES, num_classes=NUM_CLASSES).to(device)

# 确保数据也在同一设备上
data = data.to(device)

# 修改优化器设置
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 存储性能指标
accuracies = []
roc_aucs = []

from sklearn.metrics import roc_auc_score
from tqdm import trange
for epoch in trange(NUM_EPOCHS):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    # 测试模型
    model.eval()

    if epoch % 100 == 0:
        with torch.no_grad():
            logits = model(data)
            probabilities = F.softmax(logits, dim=1)
            preds = probabilities.argmax(dim=1)
            correct = preds[data.valid_mask].eq(data.y[data.valid_mask]).sum().item()
            accuracy = correct / data.valid_mask.sum().item()

            accuracies.append(accuracy)
        # 计算ROC-AUC
        y_true = data.y[data.valid_mask].cpu().numpy()  # 将数据移回CPU来计算ROC-AUC
        y_score = probabilities[data.valid_mask, 1].cpu().numpy()
        roc_auc = roc_auc_score(y_true, y_score)
        roc_aucs.append(roc_auc)
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy}, ROC-AUC: {roc_auc}')


  2%|▏         | 11/500 [00:00<00:09, 50.66it/s]

Epoch 1, Loss: 4.100464820861816, Accuracy: 0.8227956989247311, ROC-AUC: 0.5126491456006171


 22%|██▏       | 110/500 [00:02<00:08, 44.31it/s]

Epoch 101, Loss: 0.4316958785057068, Accuracy: 0.8210752688172043, ROC-AUC: 0.6946727551398455


 42%|████▏     | 210/500 [00:03<00:03, 77.24it/s]

...

 63%|██████▎   | 317/500 [00:04<00:02, 81.08it/s]

Epoch 301, Loss: 0.40675121545791626, Accuracy: 0.8356989247311828, ROC-AUC: 0.7039037956952685


 83%|████████▎ | 416/500 [00:06<00:01, 80.44it/s]

Epoch 401, Loss: 0.4013436436653137, Accuracy: 0.8365591397849462, ROC-AUC: 0.7117810433467486


100%|██████████| 500/500 [00:07<00:00, 69.64it/s]


In [ ]:
print(f'Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy}, ROC-AUC: {roc_auc}')

Epoch 500, Loss: 0.3915436863899231, Accuracy: 0.8365591397849462, ROC-AUC: 0.7117810433467486


In [ ]:
score_valid_gnn = probabilities[data.valid_mask, 1].cpu().numpy()
score_test_gnn = probabilities[data.test_mask, 1].cpu().numpy()

In [ ]:
model_gnn, y_score_gnn = model, y_score

# LSTM

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3, dropout_rate=0.8):
        super(LSTMModel, self).__init__()
        # 定义三层LSTM，每层之间添加dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout_rate)
        # Dropout层
        self.dropout = nn.Dropout(dropout_rate)
        # 定义线性层
        self.fc_token = nn.Linear(hidden_size, output_size)  # 用于 token 改变量的预测
        self.fc_label = nn.Linear(hidden_size, 1)  # 用于每个时间步的标签预测

    def forward(self, x):
        # LSTM 层
        lstm_out, _ = self.lstm(x)
        # 应用dropout
        lstm_out = self.dropout(lstm_out)
        # 在每个时间步预测 token 的改变量
        token_change = self.fc_token(lstm_out)
        # 在每个时间步预测标签
        labels = self.fc_label(lstm_out)
        return token_change, labels

In [ ]:
from sklearn.metrics import roc_auc_score
import torch
import numpy as np

def evaluate_model(model, eval_dataset):
    model.eval()  # 设置模型为评估模式

    # 确保模型在 GPU 上
    if torch.cuda.is_available():
        model = model.cuda()

    predictions = []
    true_labels = []

    with torch.no_grad():  # 在评估过程中不计算梯度
        for features, label in eval_dataset:
            # 将数据移至 GPU
            features = features.cuda() if torch.cuda.is_available() else features
            label = label.cuda() if torch.cuda.is_available() else label

            # 前向传播
            _, label_pred = model(features.unsqueeze(0))

            # 将预测转换为二元概率（sigmoid 函数）
            label_prob = torch.sigmoid(label_pred).squeeze().cpu().numpy()[-1]
            predictions.append(label_prob)

            # 保存真实标签
            true_labels.append(label.cpu().numpy())

    # 计算 ROC-AUC 分数
    roc_auc = roc_auc_score(true_labels, predictions)
    return roc_auc, predictions

In [ ]:
import torch
import torch.optim as optim
from sklearn.metrics import roc_auc_score

def train_model(train_dataset, valid_dataset, num_epochs):
    # 创建模型实例
    model = LSTMModel(input_size=171, hidden_size=64, output_size=171)

    # 检查 CUDA 是否可用，并将模型移至 GPU
    if torch.cuda.is_available():
        model = model.cuda()

    # 定义优化器和损失函数
    criterion_token = nn.MSELoss()
    criterion_label = nn.BCEWithLogitsLoss(reduction='none')
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

    best_roc_auc = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        total_loss = 0
        from tqdm import tqdm
        for features, label in tqdm(train_dataset):
            features = features.cuda() if torch.cuda.is_available() else features
            label = label.cuda() if torch.cuda.is_available() else label

            optimizer.zero_grad()
            token_change_pred, label_pred = model(features.unsqueeze(0))
            label_pred = label_pred[-1]

            actual_token_change = features[1:, :] - features[:-1, :]
            loss_token = criterion_token(token_change_pred[:, :-1], actual_token_change.unsqueeze(0))
            label = label.float().unsqueeze(0).unsqueeze(-1).expand_as(label_pred)
            loss_label = criterion_label(label_pred, label).mean()
            loss = loss_label * 10

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_dataset)}")

        # 验证步骤
        current_roc_auc, _ = evaluate_model(model, valid_dataset)
        print(f"Validation ROC-AUC for Epoch {epoch+1}: {current_roc_auc}")

        # 检查是否需要更新最佳模型
        if current_roc_auc > best_roc_auc:
            best_roc_auc = current_roc_auc
            best_model = model.state_dict()

    # 保存最佳模型
    if best_model is not None:
        torch.save(best_model, 'best_model.pth')
        print("Best model saved.")

    return best_model, best_roc_auc, model

# 使用
num_epochs = 10
best_model, best_roc_auc, model = train_model(train_dataset, valid_dataset, num_epochs)
best_roc_auc

100%|██████████| 9271/9271 [00:32<00:00, 285.66it/s]


Epoch 1/10, Loss: 4.488138884931189
Validation ROC-AUC for Epoch 1: 0.7376877166449283


100%|██████████| 9271/9271 [00:30<00:00, 307.38it/s]


Epoch 2/10, Loss: 4.278274439033368
Validation ROC-AUC for Epoch 2: 0.7469568207309213


100%|██████████| 9271/9271 [00:30<00:00, 306.94it/s]


...

100%|██████████| 9271/9271 [00:30<00:00, 304.64it/s]


Epoch 9/10, Loss: 3.8483108185828794
Validation ROC-AUC for Epoch 9: 0.7182994229568767


100%|██████████| 9271/9271 [00:30<00:00, 303.37it/s]


Epoch 10/10, Loss: 3.7867991128598275
Validation ROC-AUC for Epoch 10: 0.704698054699831
Best model saved.


0.7469568207309213

In [ ]:
from sklearn.metrics import roc_auc_score
import torch
import numpy as np

def inference_model(model, eval_dataset):
    model.eval()  # 设置模型为评估模式

    # 确保模型在 GPU 上
    if torch.cuda.is_available():
        model = model.cuda()

    predictions = []
    true_labels = []

    with torch.no_grad():  # 在评估过程中不计算梯度
        for features, label in eval_dataset:
            # 将数据移至 GPU
            features = features.cuda() if torch.cuda.is_available() else features
            label = label.cuda() if torch.cuda.is_available() else label

            # 前向传播
            _, label_pred = model(features.unsqueeze(0))

            # 将预测转换为二元概率（sigmoid 函数）
            label_prob = torch.sigmoid(label_pred).squeeze().cpu().numpy()[-1]
            predictions.append(label_prob)

            # 保存真实标签
            true_labels.append(label.cpu().numpy())

    # 计算 ROC-AUC 分数
    # roc_auc = roc_auc_score(true_labels, predictions)
    return predictions

# 使用
# valid_dataset 应该是一个适当的 DataLoader
valid_output_lstm_last = inference_model(model, valid_dataset)
test_output_lstm_last = inference_model(model, test_dataset)
roc_auc_score(valid_labels, valid_output_lstm_last)

0.7865460644846959

In [ ]:
model.load_state_dict(best_model)
valid_output_lstm_best = inference_model(model, valid_dataset)
test_output_lstm_best = inference_model(model, test_dataset)
roc_auc_score(valid_labels, valid_output_lstm_best)

0.7865460644846959

In [ ]:
lstm_full = valid_output_lstm_last+test_output_lstm_last

In [ ]:
lstm_sub = valid_output_lstm_last+test_output_lstm_last

In [ ]:
valid_ids = df_valid['id'].tolist()
test_ids = df_test['id'].tolist()
combined_ids = valid_ids + test_ids
# Removing adjacent duplicates
unique_ids = [combined_ids[i] for i in range(len(combined_ids)) if i == 0 or combined_ids[i] != combined_ids[i-1]]
dict_outputDF = {
    "id": unique_ids,
    "subset": ["valid"] * len(valid_labels)+["test"]*len(test_labels),
    "label": valid_labels.tolist()+test_labels.tolist(),
    "lstm_full": lstm_full,
    "lstm_sub": lstm_sub,
}
import pandas as pd
df = pd.DataFrame(dict_outputDF)
df.to_csv('/content/drive/MyDrive/lstm_qwq.csv', index=False)

In [ ]:
valid_ids = df_valid['id'].tolist()
test_ids = df_test['id'].tolist()
combined_ids = valid_ids + test_ids
# Removing adjacent duplicates
unique_ids = [combined_ids[i] for i in range(len(combined_ids)) if i == 0 or combined_ids[i] != combined_ids[i-1]]
dict_outputDF = {
    "id": unique_ids,
    "subset": ["valid"] * len(valid_labels)+["test"]*len(test_labels),
    "label": valid_labels.tolist()+test_labels.tolist(),
    # "lstm_pred_1": valid_output_lstm_last+test_output_lstm_last,
    # "lstm_pred_2": valid_output_lstm_best+test_output_lstm_best,
}
valid_sum = np.array(valid_output_lstm_last)-valid_output_lstm_last
for i in range(5):
    best_model, best_roc_auc, model = train_model(train_dataset, valid_dataset, 1)
    valid_output_lstm_last = inference_model(model, valid_dataset)
    test_output_lstm_last = inference_model(model, test_dataset)
    # roc_auc_score(valid_labels, valid_output_lstm_last)
    dict_outputDF[f'lstm_pred_{i}'] = valid_output_lstm_last + test_output_lstm_last
    valid_sum+=valid_output_lstm_last
roc_auc_score(valid_labels, valid_sum)

 20%|█▉        | 1836/9271 [00:08<00:34, 216.70it/s]


KeyboardInterrupt: ignored

In [ ]:
valid_sum

array([1.4353304, 1.4518121, 9.049635 , ..., 2.4956572, 6.604525 ,
       1.2105738], dtype=float32)

In [ ]:
import pandas as pd
df = pd.DataFrame(dict_outputDF)
df.to_csv('/content/drive/MyDrive/lstm_esemble.csv', index=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Setting the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Transform class
class Transform(nn.Module):
    def __init__(self, outfea, d):
        super(Transform, self).__init__()
        self.qff = nn.Linear(outfea, outfea)
        self.kff = nn.Linear(outfea, outfea)
        self.vff = nn.Linear(outfea, outfea)

        self.ln = nn.LayerNorm(outfea)
        self.lnff = nn.LayerNorm(outfea)

        self.ff = nn.Sequential(
            nn.Linear(outfea, outfea),
            nn.ReLU(),
            nn.Linear(outfea, outfea)
        )

        self.d = d

    def forward(self, x):
        query = self.qff(x)
        key = self.kff(x)
        value = self.vff(x)

        query = torch.cat(torch.split(query, self.d, -1), 0).permute(0,2,1,3)
        key = torch.cat(torch.split(key, self.d, -1), 0).permute(0,2,3,1)
        value = torch.cat(torch.split(value, self.d, -1), 0).permute(0,2,1,3)

        A = torch.matmul(query, key)
        A /= (self.d ** 0.5)
        A = torch.softmax(A, -1)

        value = torch.matmul(A ,value)
        value = torch.cat(torch.split(value, x.shape[0], 0), -1).permute(0,2,1,3)
        value += x

        value = self.ln(value)
        x = self.ff(value) + value
        return self.lnff(x)

# PositionalEncoding class
class PositionalEncoding(nn.Module):
    def __init__(self, outfea):
        super(PositionalEncoding, self).__init__()
        self.outfea = outfea if outfea % 2 == 0 else outfea + 1

    def forward(self, x):
        max_len = x.size(1)
        pe = torch.zeros(max_len, self.outfea).to(x.device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.outfea, 2).float() * -(math.log(10000.0) / self.outfea))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).unsqueeze(2)  # [1, T, 1, F]
        x = x + pe[:,:,:,:x.size(-1)]  # Adjust positional encoding size to match input size
        return x

# SGNN class
class SGNN(nn.Module):
    def __init__(self, outfea):
        super(SGNN, self).__init__()
        self.ff = nn.Sequential(
            nn.Linear(outfea, outfea),
            nn.Linear(outfea, outfea)
        )
        self.ff1 = nn.Linear(outfea, outfea)

    def forward(self, x):
        p = self.ff(x)
        a = torch.matmul(p, p.transpose(-1,-2))
        R = torch.relu(torch.softmax(a, -1)) + torch.eye(x.shape[1]).to(device)

        D = (R.sum(-1) ** -0.5)
        D[torch.isinf(D)] = 0.
        D = torch.diag_embed(D)

        A = torch.matmul(torch.matmul(D, R), D)
        x = torch.relu(self.ff1(torch.matmul(A, x)))
        return x

# GRU class
class GRU(nn.Module):
    def __init__(self, outfea):
        super(GRU, self).__init__()
        self.ff = nn.Linear(2*outfea, 2*outfea)
        self.zff = nn.Linear(2*outfea, outfea)
        self.outfea = outfea

    def forward(self, x, xh):
        r, u = torch.split(torch.sigmoid(self.ff(torch.cat([x, xh], -1))), self.outfea, -1)
        z = torch.tanh(self.zff(torch.cat([x, r*xh], -1)))
        x = u * z + (1-u) * xh
        return x


class STGNNwithGRU(nn.Module):
    def __init__(self, outfea):
        super(STGNNwithGRU, self).__init__()
        self.sgnn = SGNN(outfea)  # 使用单个SGNN实例
        self.gru = GRU(outfea)

    def forward(self, x):
        B, T, N, F = x.shape
        hidden_state = torch.zeros([B, N, F]).to(device)
        output = []

        for i in range(T):
            gx = self.sgnn(x[:, i, :, :])  # 在每个时间步使用相同的SGNN实例
            gh = hidden_state
            if i != 0:
                gh = self.sgnn(hidden_state)  # 重复使用SGNN实例
            hidden_state = self.gru(gx, gh)
            output.append(hidden_state)

        output = torch.stack(output, 1)
        return output


# Continuing with the STGNNwithGRU class
class STGNNwithGRU(nn.Module):
    def __init__(self, outfea):
        super(STGNNwithGRU, self).__init__()
        self.sgnn = SGNN(outfea)  # Using a single SGNN instance
        self.gru = GRU(outfea)

    def forward(self, x):
        B, T, N, F = x.shape
        hidden_state = torch.zeros([B, N, F]).to(device)
        output = []

        for i in range(T):
            gx = self.sgnn(x[:, i, :, :])  # Use the same SGNN instance at each time step
            gh = hidden_state
            if i != 0:
                gh = self.sgnn(hidden_state)  # Reuse the SGNN instance
            hidden_state = self.gru(gx, gh)
            output.append(hidden_state)

        output = torch.stack(output, 1)
        return output

# STGNN class
class STGNN(nn.Module):
    def __init__(self, infea, outfea, L, d):
        super(STGNN, self).__init__()
        self.start_emb = nn.Linear(infea, outfea)
        self.adjust_emb = nn.Linear(outfea, infea)  # Additional layer for dimension adjustment
        self.end_emb = nn.Linear(infea, infea)

        self.stgnnwithgru = nn.ModuleList([STGNNwithGRU(outfea) for i in range(L)])
        self.positional_encoding = PositionalEncoding(outfea)
        self.transform = nn.ModuleList([Transform(outfea, d) for i in range(L)])

        self.fc_token_change = nn.Linear(infea, infea)  # Predicts next time step change
        self.fc_label = nn.Linear(infea, 1)  # Predicts label

        self.L = L

    def forward(self, x):
        B, T, F = x.shape
        x = x.unsqueeze(2)  # Add spatial dimension [B, T, 1, F]
        x = self.start_emb(x)
        for i in range(self.L):
            x = self.stgnnwithgru[i](x)
        x = self.positional_encoding(x)
        for i in range(self.L):
            x = self.transform[i](x)
        x = self.adjust_emb(x)  # Adjust dimensions
        x = self.end_emb(x)
        x = x.squeeze(2)  # Remove spatial dimension [B, T, F]

        token_change = self.fc_token_change(x)
        labels = self.fc_label(x).squeeze(-1)

        return token_change, labels

# Instantiate and test the model
model = STGNN(infea=171, outfea=64, L=3, d=8).to(device)
test_input = torch.rand(1, 13, 171).to(device)
token_change, labels = model(test_input)

# Check the shapes of the outputs
token_change.shape, labels.shape

(torch.Size([1, 13, 171]), torch.Size([1, 13]))

In [ ]:
num_epochs = 10
best_model = train_model(model, train_dataset, valid_dataset, num_epochs)

KeyboardInterrupt: ignored

# Save all

In [ ]:
valid_labels

array([0, 0, 1, ..., 1, 0, 0])

In [ ]:
valid_ids = df_valid['id'].tolist()
test_ids = df_test['id'].tolist()
combined_ids = valid_ids + test_ids
# Removing adjacent duplicates
unique_ids = [combined_ids[i] for i in range(len(combined_ids)) if i == 0 or combined_ids[i] != combined_ids[i-1]]

In [ ]:
valid_ids = df_valid['id'].tolist()
test_ids = df_test['id'].tolist()
combined_ids = valid_ids + test_ids
# Removing adjacent duplicates
unique_ids = [combined_ids[i] for i in range(len(combined_ids)) if i == 0 or combined_ids[i] != combined_ids[i-1]]
dict_outputDF = {
    "id": unique_ids,
    "subset": ["valid"] * len(valid_labels)+["test"]*len(test_labels),
    "label": valid_labels.tolist()+test_labels.tolist(),
    "gnn_pred": score_valid_gnn.tolist()+score_test_gnn.tolist(),
    "lstm_pred_1": valid_output_lstm_last+test_output_lstm_last,
    "lstm_pred_2": valid_output_lstm_best+test_output_lstm_best,
}
import pandas as pd
df = pd.DataFrame(dict_outputDF)
df.to_csv('/content/drive/MyDrive/cogito/gnn_lstm.csv', index=False)

In [ ]:
len(dict_outputDF['id'])

5261

In [ ]:
len(dict_outputDF['gnn_pred'])

5261

In [ ]:
import pandas as pd
df = pd.DataFrame(dict_outputDF)
df.to_csv('/gdrive/MyDrive/gnn_lstm.csv', index=False)